In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import libraries
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess


def formatting_for_url(string):
    return string.replace(" ", "+")

search_keywords = formatting_for_url(input("Please enter your search keywords : "))
search_location = formatting_for_url(input("Please enter the name of the city : "))

# Example manually entered keywords
# search_keywords = "restaurant+japonais"
# search_location = "Bordeaux"

In [2]:
class YelpSpider(scrapy.Spider):

    # Name of your spider
    name = "yelp"

    # Url to start your spider from 
    start_urls = [
        'https://www.yelp.fr/search?find_desc={}&find_loc={}'.format(search_keywords,search_location),
    ]

    def parse(self, response):
        
        paragraphs = response.css('a.css-1m051bw')
        #page_number = response.css(' li > div > div > span.css-chan6m::text').getall()[0].split()[2]
        
        for p in paragraphs[3:]: 
            yield {
                'text': p.css('::text').getall()
            }
        
        try:
            # Select the NEXT button and store it in next_page
            next_page = response.css('a.next-link').attrib["href"]
        except KeyError:
            # In the last page, there won't be any "href" and a KeyError will be raised
            logging.info('No next page. Terminating crawling process.')
        else:
            # If a next page is found, execute the parse method once again
            yield response.follow(next_page, callback=self.parse)
            
    

In [3]:
# Result file
filename = "{}-{}.json".format(search_keywords,search_location)

# Declare a new CrawlerProcess 
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'results/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(YelpSpider)
process.start()

2022-04-17 02:31:13 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-17 02:31:13 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.0 (v3.9.0:9cf6752276, Oct  5 2020, 11:29:23) - [Clang 6.0 (clang-600.0.57)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.2, Platform macOS-10.16-x86_64-i386-64bit
2022-04-17 02:31:13 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-04-17 02:31:13 [scrapy.extensions.telnet] INFO: Telnet Password: ec35031f7834a279
2022-04-17 02:31:13 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-04-17 02:31:13 [scrapy.middleware] INFO: Enabled dow